<a href="https://colab.research.google.com/github/deepakgowtham/EVA4/blob/master/Week14/Session15A_Mask_RCNN_CV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import os
from skimage import io
from tqdm import tqdm


In [0]:
#copy bg/fg/mask files from Drive
%%bash
cd /content
mkdir Mask_RCNN
cp -r /content/gdrive/My\ Drive/RCNN/* Mask_RCNN/
#cp -r /content/gdrive/My\ Drive/resized/Cars_Mask Mask_RCNN/
#cp -r /content/gdrive/My\ Drive/resized/bg Mask_RCNN/
#cp -r /content/gdrive/My\ Drive/resized/fg_bg Mask_RCNN/
#cp -r /content/gdrive/My\ Drive/resized/fg_bg_mask Mask_RCNN/
ls Mask_RCNN/fg/ |wc -l
ls Mask_RCNN/Cars_Mask/ |wc -l
ls Mask_RCNN/bg/ |wc -l
ls Mask_RCNN/fg_bg/ |wc -l
ls Mask_RCNN/fg_bg_mask/ |wc -l

ls -lrt Mask_RCNN/

104
0
102
0
0
total 20
drwx------ 2 root root 4096 May  9 06:26 bg
drwx------ 2 root root 4096 May  9 06:26 Cars_Mask
drwx------ 2 root root 4096 May  9 06:26 fg
drwx------ 2 root root 4096 May  9 06:26 fg_bg_mask
drwx------ 2 root root 4096 May  9 06:26 fg_bg


mkdir: cannot create directory ‘Mask_RCNN’: File exists


In [0]:
#Function to gegerate Mask
def gen_mask(T, image):
    # grab the image dimensions
    h = image.shape[0]
    w = image.shape[1]
    
    # loop over the image, pixel by pixel
    for y in range(0, h):
        for x in range(0, w):
            # threshold the pixel
            image[y, x] = 255 if image[y, x] >= T else 0
            
    # return the thresholded image
    return image

In [0]:
#Generate mask
import os
import numpy as np
import matplotlib.pyplot as plt

path='./Mask_RCNN/fg/'
tgt_path='./Mask_RCNN/Cars_Mask/'
for im_name in os.listdir(path):
    src1 = cv2.imread(path+im_name)
    gray = cv2.cvtColor(src1, cv2.COLOR_BGR2GRAY)
    img=gen_mask(1,gray)
    arr = np.asarray(img)
    cv2.imwrite(tgt_path+im_name, img)

In [0]:
!cp Mask_RCNN/Cars_Mask/* /content/gdrive/My\ Drive/RCNN/Cars_Mask/

In [0]:
## Function to Create Overlay
def overlay_fg_bg(fg,bg,x_offset,y_offset,tgt_save):
    fg_bg=bg.copy()
    y1, y2 = y_offset, y_offset + fg.shape[0]
    x1, x2 = x_offset, x_offset + fg.shape[1]
    
    alpha_fg = fg[:, :, 3] / 255.0
    alpha_bg = 1.0 - alpha_fg

    for c in range(0, 3):
        fg_bg[y1:y2, x1:x2, c] = (alpha_fg * fg[:, :, c] +
                                  alpha_bg * bg[y1:y2, x1:x2, c])


    cv2.imwrite(tgt_save,fg_bg)
    
    #arr = np.asarray(fg_bg)
    #plt.imshow(arr, vmin=0, vmax=255)
    #plt.show()
    
def overlay_mask(fg,bg,x_offset,y_offset,tgt_save):
    fg_bg=bg.copy()
    fg_bg[y_offset:y_offset+fg.shape[0], x_offset:x_offset+fg.shape[1]] = fg


    cv2.imwrite(tgt_save,fg_bg)

In [0]:
%%timeit
#Generate fg_bg and mask with fg Size
tgt_path="./Mask_RCNN/fg_bg/"
mask_tgt_path="./Mask_RCNN/fg_bg_mask/"
bg_path="./Mask_RCNN/bg/"
fg_path="./Mask_RCNN/fg/"
mask_path="./Mask_RCNN/Cars_Mask/"

mask_bg = np.zeros((224,224), np.uint8)

for i_bg in os.listdir(bg_path):
    bg = cv2.imread(bg_path+i_bg)
    bg_name=os.path.splitext(i_bg)[0]
    
    for i_fg in os.listdir(fg_path):
        fg = cv2.imread(fg_path+i_fg, -1)
        fg_name=os.path.splitext(i_fg)[0]
        mask_fg = cv2.imread(mask_path+i_fg, -1)
        fg_flip=cv2.flip(fg,1)
        fg_flip_name=fg_name+'_flip'
        mask_fg_flip=cv2.flip(mask_fg,1)
        for i in range(1,21):
            x_offset=randint(0,64)
            y_offset=randint(0,64)
            #fg on bg
            tgt_save=tgt_path+bg_name+fg_name+'_'+str(i)+'.jpg'
            overlay_fg_bg(fg,bg,x_offset,y_offset,tgt_save)
            
            #mask on black bg
            tgt_save=mask_tgt_path+bg_name+fg_name+'_'+str(i)+'.jpg'
            overlay_mask(mask_fg,mask_bg,x_offset,y_offset,tgt_save)

            #LR Flipped fg on bg
            tgt_save=tgt_path+bg_name+fg_flip_name+'_'+str(i)+'.jpg'
            overlay_fg_bg(fg_flip,bg,x_offset,y_offset,tgt_save)
            
            #LR Flipped mask on black bg
            tgt_save=mask_tgt_path+bg_name+fg_flip_name+'_'+str(i)+'.jpg'
            overlay_mask(mask_fg_flip,mask_bg,x_offset,y_offset,tgt_save)

In [0]:
%%bash
#Check the image counts
cd /content
ls Mask_RCNN/fg/ |wc -l
ls Mask_RCNN/Cars_Mask/ |wc -l
ls Mask_RCNN/bg/ |wc -l
ls Mask_RCNN/fg_bg/ |wc -l
ls Mask_RCNN/fg_bg_mask/ |wc -l
du -sh Mask_RCNN/

104
104
102
424320
424320
11G	Mask_RCNN/


In [0]:
#Zip files to Google Drive
%%bash
zip -r /content/gdrive/My\ Drive/RCNN/fg_bg.zip ./Mask_RCNN/fg_bg
zip -r /content/gdrive/My\ Drive/RCNN/fg_bg_mask.zip ./Mask_RCNN/fg_bg_mask

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

